In [ ]:
!pip install fastkml
!pip install shapely
!pip install geopy

In [ ]:
import csv, json
import requests, time
import urllib
import pandas as pd
from geopy.distance import great_circle

# from shapely.geometry import Point, Polygon

# https://api.telegram.org/bot<your-bot-token>/getUpdates
# https://api.telegram.org/bot<your-bot-id>/sendMessage?chat_id=<chat-id>&text=TestReply

TOKEN = "6182927778:AAH9kwbB0D_lSGzDG-lvoMIo2ZzrNK3IoAM"
URL = "https://api.telegram.org/bot{}/".format(TOKEN)

def get_url(url):
    response = requests.get(url)
    content = response.content.decode("utf8")
    return content


def get_json_from_url(url):
    content = get_url(url)
    js = json.loads(content)
    return js


def get_updates(offset=None):
    url = URL + "getUpdates"
    if offset:
        url += "?offset={}".format(offset)
    js = get_json_from_url(url)
    return js

def get_last_update_id(updates):
    update_ids = []
    for update in updates["result"]:
        update_ids.append(int(update["update_id"]))
    return max(update_ids)

def fuel_nearby(updates,chat):
    for update in updates["result"]:
        locations = pd.read_csv("palermo_carburanti.csv", sep=";")
        prices = pd.read_csv("prezzo_alle_8.csv", sep=";")
                    
        table = locations.join(prices, lsuffix="idImpianto", rsuffix="idImpianto")
        
        print(update["update_id"])
        
        if (update["message"]["location"] > 0):
            position = update["message"]["location"]
        else:
            continue
            
        mypos = (position["latitude"],position["longitude"])
        chat = update["message"]["chat"]["id"]

        for i in table.index:
            lat = table["Latitudine"][i]
            lon = table["Longitudine"][i]
            banner = table["Bandiera"][i]
            fuel = table["descCarburante"][i]
            price = str(table["prezzo"][i])
            
            dist = great_circle(mypos, [lat,lon]).meters
            if  dist < 1000:
                send_location(chat,lat,lon)
                message = banner + ":\n" + fuel + " " + price 
                send_message(message,chat)

def get_last_chat_id(updates):
    last_update = len(updates["result"])
    chat_id = updates["result"][last_update-1]["message"]["chat"]["id"]
    return chat_id

def send_message(text, chat_id):
    text = urllib.parse.quote_plus(text)
    url = URL + "sendMessage?text={}&chat_id={}".format(text, chat_id)
    get_url(url)
    
def send_location(chat_id, latitude, longitude):
    # text = urllib.parse.quote_plus(text)
    url = URL + "sendLocation?chat_id={}&latitude={}&longitude={}".format(chat_id, latitude, longitude)
    get_url(url)

def main():
    last_update_id = None
    while True:
        updates = get_updates(last_update_id)
        if len(updates["result"]) > 0:
            chat = get_last_chat_id(updates)
            last_update_id = get_last_update_id(updates) + 1
            fuel_nearby(updates,chat)
        time.sleep(0.5)

if __name__ == '__main__':
    main()